In [ ]:
path = []


In [ ]:
path = [[0, 10], [1, 9], [2, 9], [3, 10], [4, 9], [5, 10], [6, 9], [7, 9], [8, 10], [9, 10], [10, 11], [11, 10], [12, 10], [13, 11],[14, 11], [15, 11], [16, 10], [17, 10], [18, 9], [19, 10]]

In [3]:
import cv2
import numpy as np
blank = cv2.imread("blank.png",0)
blank = cv2.resize(blank,(20,20))
# path = np.array(path)
# epsilon = 0.1*cv2.arcLength(path, True)
# path = cv2.approxPolyDP(path, epsilon, False)
# print(path)
pnts = []
for i in range(1,16):
    pts = [np.mean(path[0:15*(i)][0]),np.mean(path[0:15*(i)][1])]
    pnts.append(pts)
#     cv2.line(blank,(path[i][1],path[i][0]),(path[i+1][1],path[i+1][0]),0)
#     pnts = np.mean(path[i:i+15][0])
print(pnts)
blank = cv2.resize(blank,(680,480))
cv2.imshow("blank",blank)
q = cv2.waitKey(1)
if q == 27:
    cv2.destroyAllWindows()

NameError: name 'path' is not defined

In [3]:
import math
slp = []
for pix in pnts:
    slp.append(math.atan2(340-pix[1],pix[0]-240) - 1.57)

NameError: name 'pnts' is not defined

In [4]:
import math
print(slp)

[]


In [6]:
#output:: 6 category  
#input:: 15 slope 
#mode = xgboost
minibatch = []

In [5]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import time
import threading

In [8]:
import random
from collections import deque
class DQNAgent:
    def __init__(self):
        self.state_size = 15
        self.action_size = 6
        self.memory = deque(maxlen=pow(10,6))
        self.gamma = 0.8
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.99
        self.bst = xgb.Booster({'nthread':4})
        self.name = 'steerDeepQL'
        self.modelname = 'xgbmodel'
        self.load_bst()
    
    def xgbmodel_bld(self,data,label):
        param = {'base_score' : 1, 'max_depth': 7, 'eta': 0.1, #'updater':'refresh',
            #'process_type': 'update',
            'refresh_leaf': True,
            'reshape': True,
            'reg_alpha': 3, 
            'silent': 1, 
            'objective': 'multi:softprob',     
            'num_class': 6}  
        param['nthread'] = 4
        param['eval_metric'] = 'merror'
        num_round = 500  
        dtrain = xgb.DMatrix(data, label=label)
        evallist = [(dtrain, 'train'), (dtrain, 'eval')]
        self.bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=3, xgb_model = self.name)
        self.save_bst()

        
    def load_bst(self):
        self.bst.load_model(self.name)
    
    def save_bst(self):
        self.bst.save_model(self.name)
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.bst.predict(xgb.DMatrix(state))
#         print('act_values',act_values)
        return np.argmax(act_values[0])  
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self):
        minibatch = self.memory
#         print(len(minibatch))
        bst_target = xgb.Booster({'nthread':4})
        bst_target.load_model(self.name)
        st = []
        tg = []
        for state, action, reward, next_state, done in minibatch:

            target = bst_target.predict(xgb.DMatrix(state))##target is Q matrix
            if done:
                target[0][action] = reward
            else:
                t = self.bst.predict(xgb.DMatrix(next_state))[0]
                target[0][action] = (1-self.gamma)*reward + self.gamma * np.amax(t)
            st.append(state)
            tg.append(np.argmax(target[0]))
        self.memory.clear()
        st = np.array(st)
        st = np.reshape(st, [-1, 15])
        tg = np.array(tg)
        tg = np.reshape(tg, [-1,1])
        print(st.shape, tg.shape)
        self.xgbmodel_bld(st, tg)

        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def decl_rew(self,state):
        rew = (np.sum((np.array([1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75,1.75]) - abs(state))*np.array([10,10,10,9,9,9,8,8,7,6,5,4,3,2,1])))/175
#         print('reward',rew)
#         print('state',state)
        done = False
        if rew > 0.74:
            done = True
        return rew,done
    
    def get_nextstate(self):
        ########
        stt = np.random.randint(175, size=15)/100
        return stt
    
def steer(GameOn):
    
    state_size = 15
    action_size = 6
    agent = DQNAgent()
    batch_size = 50
    state = np.zeros((15,), dtype=float)
    state = agent.get_nextstate()
#     agent.load_bst()
    count = 0
    bst_target = xgb.Booster({'nthread':4})
    st = []
    tg = []
    bst_target.load_model(agent.name)
    
    while GameOn:
        start = time.time()
        count += 1
        state = np.reshape(state, [1, state_size])
        action = agent.act(state)
#         execute_action(action)
        reward, done = agent.decl_rew(state)
#         reward = reward 
        next_state = agent.get_nextstate()
        if(next_state == []):
            continue;
        next_state = np.reshape(next_state, [1, state_size])
        
#         agent.remember(state, action, reward, next_state, done)
        target = bst_target.predict(xgb.DMatrix(state))
        if done:
            target[0][action] = reward
        else:
            t = agent.bst.predict(xgb.DMatrix(next_state))[0]
            target[0][action] = (1-agent.gamma)*reward + agent.gamma * np.amax(t)
        st.append(state)
        tg.append(np.argmax(target[0]))
        
        if count % 250 == 0:
            count = 0
            st = np.array(st)
            st = np.reshape(st, [-1, 15])
            tg = np.array(tg)
            tg = np.reshape(tg, [-1,1])
            agent.xgbmodel_bld(st, tg)
            if agent.epsilon > agent.epsilon_min:
                agent.epsilon *= agent.epsilon_decay
            st = []
            tg = []
            bst_target = None
            bst_target = xgb.Booster({'nthread':4})
            bst_target.load_model(agent.name)
            print("success")
        state = next_state
#         if done and len(agent.memory) >= 50 or len(agent.memory) >= 500 :
#             print("replaying at {} xgboost".format(len(agent.memory)))
#             t1 = threading.Thread(target=agent.replay, args=())
#             t1.start()
#         time.sleep(0.03)
        print("FPS: ", 1.0 / (time.time() - start))
        

In [9]:
steer(True)

/Users/shreyashkawalkar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:118: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


FPS:  52.82897951986296
FPS:  53.70221374339014
FPS:  52.153689289000525
FPS:  51.76747056355064
FPS:  53.610920803722074
FPS:  49.64966026657828
FPS:  48.11416247963842
FPS:  47.93161611775193
FPS:  47.4629851759647
FPS:  45.394374276221086
FPS:  43.08434427997658
FPS:  31.02984390027373
FPS:  35.40064651716309
FPS:  36.283220443083415
FPS:  42.68403484490759
FPS:  50.090213172508506
FPS:  53.729042836646855
FPS:  53.29145543485166
FPS:  55.08094762830278
FPS:  48.845380754404964
FPS:  50.42079196018561
FPS:  31.636500776901144
FPS:  38.65896124245357
FPS:  35.96833917897968
FPS:  43.57311004685276
FPS:  54.00715922845149
FPS:  54.01063651699138
FPS:  54.32754779545101
FPS:  50.661352079332296
FPS:  44.41095687345806
FPS:  45.61852452062691
FPS:  52.241384034775244
FPS:  51.82695943357758
FPS:  55.36231042356886
FPS:  54.4969596174835
FPS:  54.60906699997396
FPS:  54.698800208659364
FPS:  53.952277434043815
FPS:  51.514419061655616
FPS:  46.362806327169025
FPS:  38.216196515781036
FPS

FPS:  37.792991593155584
FPS:  41.53351949775217
FPS:  46.94188089668834
FPS:  46.002281302097046
FPS:  42.78286769280986
FPS:  38.15639896655871
FPS:  34.95632026802904
FPS:  49.67435690937515
FPS:  50.44868895838345
FPS:  48.461611341552185
FPS:  41.654788861081315
FPS:  48.104229745848244
FPS:  49.54116910574867
FPS:  49.81062882251648
FPS:  51.83720786523797
FPS:  50.10756696055241
FPS:  53.610920803722074
FPS:  47.370784486458405
FPS:  51.1244865372192
FPS:  49.08718957002083
FPS:  43.58397672364524
FPS:  45.43026731944023
FPS:  49.80116597998124
FPS:  48.647660581317126
FPS:  40.761367943323066
FPS:  40.99883678875497
FPS:  49.57630344077633
FPS:  50.83080652002666
FPS:  51.18250597940157
FPS:  48.45153464946227
FPS:  45.16122919223895
FPS:  46.080618758308525
FPS:  50.30166820574937
FPS:  49.34881696139682
FPS:  50.54596288262232
FPS:  52.29283861958907
FPS:  53.43062420382166
FPS:  52.756550067293055
FPS:  53.63971660229685
FPS:  53.1948051948052
FPS:  46.62461788147934
FPS:  3

FPS:  36.288557041753904
FPS:  38.81207028972767
FPS:  34.03003578006215
FPS:  33.731183401021354
FPS:  39.456863058673015
FPS:  34.15640447242196
FPS:  32.25420066287806
FPS:  33.22484157160963
FPS:  31.15546146703807
FPS:  30.431215491659955
FPS:  33.05256190010875
FPS:  35.64826871100988
FPS:  38.41148781068557
FPS:  39.1731094320591
FPS:  50.2059323454071
FPS:  46.89779169229049
FPS:  36.508081854343835
FPS:  31.19230139961031
FPS:  32.6816998862379
FPS:  36.70584940665803
FPS:  40.400936262847125
FPS:  34.064583197972844
FPS:  32.742931193305125
FPS:  34.04578070716582
FPS:  28.971189777240546
FPS:  38.51341995317019
FPS:  39.861095007745455
FPS:  40.68505800644085
FPS:  31.14967694021537
FPS:  43.628405296607966
FPS:  47.39112356503661
FPS:  35.58324637533617
FPS:  33.153408372328315
FPS:  44.942021065715174
FPS:  51.27949824557113
FPS:  51.16439977066738
FPS:  52.49376102927373
FPS:  54.203279874904695
FPS:  54.92946384137877
FPS:  49.05331851938483
FPS:  38.30729466348832
FPS: 

KeyboardInterrupt: 

In [59]:
data = np.random.rand(100,15) # 5 entities, each contains 10 features
label = np.random.randint(6, size=100)
dtest = xgb.DMatrix([data[0]])
print(label.shape)
DQNAgent().xgbmodel_bld(data,label)
DQNAgent().bst.predict(dtest)
# np.argmax([DQNAgent().bst.predict(dtest),10])

(100,)
[0]	train-merror:0.41	eval-merror:0.41
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.33	eval-merror:0.33
[2]	train-merror:0.25	eval-merror:0.25
[3]	train-merror:0.25	eval-merror:0.25
[4]	train-merror:0.26	eval-merror:0.26
[5]	train-merror:0.25	eval-merror:0.25
[6]	train-merror:0.19	eval-merror:0.19
[7]	train-merror:0.19	eval-merror:0.19
[8]	train-merror:0.18	eval-merror:0.18
[9]	train-merror:0.16	eval-merror:0.16
[10]	train-merror:0.17	eval-merror:0.17
[11]	train-merror:0.15	eval-merror:0.15
[12]	train-merror:0.15	eval-merror:0.15
[13]	train-merror:0.13	eval-merror:0.13
[14]	train-merror:0.13	eval-merror:0.13
[15]	train-merror:0.13	eval-merror:0.13
[16]	train-merror:0.13	eval-merror:0.13
[17]	train-merror:0.12	eval-merror:0.12
[18]	train-merror:0.13	eval-merror:0.13
[19]	train-merror:0.12	eval-merror:0.12
[20]	train-merror:0.13	eval-merror:0.13
[21]	train-merro

array([ 0.5], dtype=float32)